In [7]:
cd /Users/karolinegriesbach/Documents/Innkeepr/Git/evaluation-and-execution-scripts/

In [8]:
import logging
import pandas as pd
from general_functions.return_account_ids import return_account_ids
from general_functions.call_api_with_account_id import call_api_with_accountId, send_to_innkeepr_api_paginated
from general_functions.constants import return_api_url

In [9]:
accounts = return_account_ids()

In [42]:
goals = pd.DataFrame(columns=["account","goal.name", "goal.conversionEvents"])
no_syntax_match = []
for account in accounts:
    account_id = account["id"]
    account_name = account["name"]
    print(f"Account: {account_name}")
    response = call_api_with_accountId(
        f"{return_api_url()}/goals/query",
        account_id,
        {},
        logging
    )
    if len(response) == 0:
        goals.loc[len(goals)] = [account_name, None, None]
        continue
    for entry in response:
        match_with_string = False
        conv_events = "&".join(entry["conversionEvents"])
        #print(f"conv_events: {conv_events}")
        if account_name == "Pendix":
            if "erwaegung" in conv_events.lower():
                match_with_string = True
        elif account_name == "to teach":
            if "signed_up" in conv_events.lower():
                match_with_string = True
        elif account_name == "DocMorris":
            if "order completed" in conv_events.lower():
                match_with_string = True
        elif "checkout" in conv_events.lower() and "completed" in conv_events.lower():
            match_with_string = True
        else:
            match_with_string = False
        if match_with_string:
            goals.loc[len(goals)] = [account_name, entry["name"], entry["conversionEvents"]]
goals

In [43]:
checkouts_to_ignore = ["checkout_completed_sub","checkout_completed_trial"]
goals = goals[~goals["goal.name"].isin(checkouts_to_ignore)]
goals

In [44]:
goals[["account","goal.conversionEvents"]].to_csv("goal_mapping.csv", index=False)